### Packages

In [3]:
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras_metrics import KerasMetrics
from keras.callbacks import EarlyStopping
from keras import optimizers
import numpy as np
import random
import pickle
import decoding2
import decoding_function as df
from Files import nodes
import os
import utility as ut
sys.path.append('../')

Using TensorFlow backend.


In [3]:
def pipeline(verbs):
    """
    Pipeline receives a list of verbs and decodes it.
    
    :verbs type: list
    :r type: string
    """
    import coding_function as cf
    test_list = []
    for i in verbs:
        coding = cf.coding(i)        
        test_list.append(coding)
    test_list = np.array(test_list)
    prediction = model.predict(test_list)
    for i in prediction:
        print(decoding2.decoding(i))
    return test_list.shape

In [26]:
def pipeline(verbs):
    """
    Pipeline receives a list of verbs and predicts a conjugated form for them.
    The prediction is saved in an output file.

    :verbs type: list
    :r type: string
    """
    import coding_function as cf
    from time import gmtime, strftime
    import itertools
    time = strftime("%Y-%m-%d_%H:%M:%S", gmtime())
    name = "output" + time + ".txt"

    correct = {'#pega#': '#pEgu', '#sega#': '#sEgu#', '#seka#': '#sEku#',
               '#leva#': '#lEvu#', '#ora#': '#Oru#', '#mora#': '#mOru#',
               '#posta#': '#pOtu#', '#joga#': '#jOgu#', '#sortia#': '#soiu#',
               '#media#': '#medeiu#', '#kompo#': '#koiu#', '#po#': '#poiu#', 
               '#menti#': '#mintu#', '#tosi#': '#tusu#', '#kobri#': 
               '#kubro#', '#faze#': '#fasu#', '#mata#': '#matu#', '#paga#': 
               '#pagu#', '#sai#': '#saiu#', '#bate#': '#batu#', '#kome#': 
               '#komu#'}

    f = open('Files/Results/'+ name, "w")
    f.write("train data set:" + path + '\n' + 'Results:\n' + 'model name: ' + load + '\n')
    test_list = []
    for i in verbs:
        coding = cf.coding(i)
        test_list.append(coding)
    test_list = np.array(test_list)
    prediction = model.predict(test_list)
    for i,j in list(zip(verbs, prediction)):
        f.write('verb: '+ i + ", expected: " + correct[i] + ", prediction: " + 
                decoding2.decoding(j) + '\n')
        print('verb: '+ i + ", expected: " + correct[i] + ", prediction: " + 
                decoding2.decoding(j) + '\n')
    scores = model.evaluate(X, Y)
    f.write("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100, 
                                                  model.metrics_names[2], scores[2]*100,
                                                  model.metrics_names[3], scores[3]*100 ))
    print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100, 
                                                  model.metrics_names[2], scores[2]*100,
                                                  model.metrics_names[3], scores[3]*100 ))
    
    f.close()
    return print("\nPredictions saved in the file: " + name) 

### 1. Load Data

In [5]:
path = '../data/prop_55.csv' #len 464
#path = '../data/prop_65.csv'
#path = '../data/prop_75.csv'
#path = '../data/prop_85.csv'
#path = '../data/prop_95.csv'
#path = 'Files/50_50_morar.csv'
X,Y = ut.load_data(path)
batch = len(X)

### 2. Fit Neural Net (Optional)

In [56]:
# 1. Define Model
model = Sequential()
model.add(Dense(460, input_shape=(460,), activation='sigmoid'))


# 2. Compile model
keras_metrics = KerasMetrics()
model.compile( 
    optimizer='adam', 
    loss='mean_squared_error',
    metrics = [ keras_metrics.fbeta_score,
               keras_metrics.recall,
               keras_metrics.precision]
            )

# 3. Fit model
stopper = EarlyStopping(monitor='fbeta_score', min_delta=0.00005, patience=50, verbose=1, mode='max')
model.fit(X,Y,epochs = 150, batch_size=batch,verbose=False,callbacks= [stopper])

# 4. Evaluate model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100, 
                                                  model.metrics_names[2], scores[2]*100,
                                                  model.metrics_names[3], scores[3]*100 ))

# 5. Save model
model.save('../data/50_50_morar')

### 2. (Or Load Trained Model)
For example: 

In [4]:
#model = load_model('../data/prop_55',custom_objects={'fbeta_score': keras_metrics.fbeta_score,

#### 2.1 Train existing model

In [ ]:
#model.fit(X,Y,epochs = 150, batch_size=batch,verbose=False,callbacks= [stopper])

### 3. Check Results
Test the prediction for some verbs.

pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#tendi#','#jenti#',
          '#menti#','#hendi#','#tosi#','#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])
    

|       	|       	| front 	| middle  	| back    	|
|:-----:	|-------	|------:	|---------	|---------	|
|       	|       	| v/u   	| v/u     	| v/u     	|
|  **int**  | **stop**  | b/p   	| d/t     	| g/k     	|
|       	| **nasal** | m/-   	| n/-     	| -       	|
| **cont**  | **fric**  | v/f   	| z/s    	| j/x   	|
|       	| **liq**   | l/-   	| r/-     	| -/h     	|
| **vowel** | **high**  | e/i   	| -       	| o/u     	|
|       	| **low**   | -/E   	| -/a     	| -/O     	|

## Results:

## Proporcao 55% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_55         | prop_55       |        209            |  464        |    55%     | 150    |  464         |

In [27]:
keras_metrics = KerasMetrics()
load = '../data/prop_55'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu, prediction: Resposta:#begu#

verb: #sega#, expected: #sEgu#, prediction: Resposta:#segu#

verb: #seka#, expected: #sEku#, prediction: Resposta:#seku#

verb: #leva#, expected: #lEvu#, prediction: Resposta:#lesu#

verb: #ora#, expected: #Oru#, prediction: Resposta:#erru#

verb: #mora#, expected: #mOru#, prediction: Resposta:#muru#

verb: #posta#, expected: #pOtu#, prediction: Resposta:#butu#

verb: #joga#, expected: #jOgu#, prediction: Resposta:#xogu#

verb: #sortia#, expected: #soiu#, prediction: Resposta:#suiu#

verb: #media#, expected: #medeiu#, prediction: Resposta:#meiu#

verb: #kompo#, expected: #koiu#, prediction: Resposta:#koiu#

verb: #po#, expected: #poiu#, prediction: Resposta:#poiu#

verb: #menti#, expected: #mintu#, prediction: Resposta:#metu#

verb: #tosi#, expected: #tusu#, prediction: Resposta:#tofu#

verb: #kobri#, expected: #kubro#, prediction: Resposta:#koru#

verb: #faze#, expected: #fasu#, prediction: Resposta:#fasu#

verb: #mata#, expe

## Proporcao 65% Irregulares


|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_65     | prop_65       |      209        |  579        |    65%     | 150    |  579         |

In [22]:
keras_metrics = KerasMetrics()
load = '../data/prop_65'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu, prediction: Resposta:#bexu#

verb: #sega#, expected: #sEgu#, prediction: Resposta:#segu#

verb: #seka#, expected: #sEku#, prediction: Resposta:#seku#

verb: #leva#, expected: #lEvu#, prediction: Resposta:#levu#

verb: #ora#, expected: #Oru#, prediction: Resposta:#erru#

verb: #mora#, expected: #mOru#, prediction: Resposta:#buru#

verb: #posta#, expected: #pOtu#, prediction: Resposta:#bOtu#

verb: #joga#, expected: #jOgu#, prediction: Resposta:#xogu#

verb: #sortia#, expected: #soiu#, prediction: Resposta:#suiu#

verb: #media#, expected: #medeiu#, prediction: Resposta:#meiu#

verb: #kompo#, expected: #koiu#, prediction: Resposta:#koiu#

verb: #po#, expected: #poiu#, prediction: Resposta:#poiu#

verb: #menti#, expected: #mintu#, prediction: Resposta:#mitu#

verb: #tosi#, expected: #tusu#, prediction: Resposta:#tusu#

verb: #kobri#, expected: #kubro#, prediction: Resposta:#koru#

verb: #faze#, expected: #fasu#, prediction: Resposta:#fasu#

verb: #mata#, expe

## Proporcao 75% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_75       | prop_75       |            300          |  400        |    75%     | 150   |  400         |

In [23]:
keras_metrics = KerasMetrics()
load = '../data/prop_75'
model = load_model(load, custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu, prediction: Resposta:#beku#

verb: #sega#, expected: #sEgu#, prediction: Resposta:#segu#

verb: #seka#, expected: #sEku#, prediction: Resposta:#seku#

verb: #leva#, expected: #lEvu#, prediction: Resposta:#levu#

verb: #ora#, expected: #Oru#, prediction: Resposta:#irru#

verb: #mora#, expected: #mOru#, prediction: Resposta:#miru#

verb: #posta#, expected: #pOtu#, prediction: Resposta:#putu#

verb: #joga#, expected: #jOgu#, prediction: Resposta:#xogu#

verb: #sortia#, expected: #soiu#, prediction: Resposta:#suiu#

verb: #media#, expected: #medeiu#, prediction: Resposta:#meiu#

verb: #kompo#, expected: #koiu#, prediction: Resposta:#koiu#

verb: #po#, expected: #poiu#, prediction: Resposta:#poiu#

verb: #menti#, expected: #mintu#, prediction: Resposta:#mitu#

verb: #tosi#, expected: #tusu#, prediction: Resposta:#tosu#

verb: #kobri#, expected: #kubro#, prediction: Resposta:#koru#

verb: #faze#, expected: #fasu#, prediction: Resposta:#fasu#

verb: #mata#, expe

## Proporcao 85% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_85       | prop_85       |             340         |  400        |    85%     | 150   |  400         |

In [24]:
keras_metrics = KerasMetrics()
load = '../data/prop_85'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu, prediction: Resposta:#beku#

verb: #sega#, expected: #sEgu#, prediction: Resposta:#segu#

verb: #seka#, expected: #sEku#, prediction: Resposta:#siku#

verb: #leva#, expected: #lEvu#, prediction: Resposta:#lesu#

verb: #ora#, expected: #Oru#, prediction: Resposta:#iaru#

verb: #mora#, expected: #mOru#, prediction: Resposta:#biru#

verb: #posta#, expected: #pOtu#, prediction: Resposta:#bOtu#

verb: #joga#, expected: #jOgu#, prediction: Resposta:#xogu#

verb: #sortia#, expected: #soiu#, prediction: Resposta:#suiu#

verb: #media#, expected: #medeiu#, prediction: Resposta:#meiu#

verb: #kompo#, expected: #koiu#, prediction: Resposta:#koiu#

verb: #po#, expected: #poiu#, prediction: Resposta:#poiu#

verb: #menti#, expected: #mintu#, prediction: Resposta:#mitu#

verb: #tosi#, expected: #tusu#, prediction: Resposta:#tofu#

verb: #kobri#, expected: #kubro#, prediction: Resposta:#koru#

verb: #faze#, expected: #fasu#, prediction: Resposta:#fasu#

verb: #mata#, expe

## Proporcao 95% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_95       | prop_95       |             380         |  400        |    95%     | 150   |  400         |

In [25]:
keras_metrics = KerasMetrics()
load = '../data/prop_95'
model = load_model(load, custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

verb: #pega#, expected: #pEgu, prediction: Resposta:#pEku#

verb: #sega#, expected: #sEgu#, prediction: Resposta:#sigu#

verb: #seka#, expected: #sEku#, prediction: Resposta:#siku#

verb: #leva#, expected: #lEvu#, prediction: Resposta:#lesu#

verb: #ora#, expected: #Oru#, prediction: Resposta:#earu#

verb: #mora#, expected: #mOru#, prediction: Resposta:#mOru#

verb: #posta#, expected: #pOtu#, prediction: Resposta:#pOtu#

verb: #joga#, expected: #jOgu#, prediction: Resposta:#xogu#

verb: #sortia#, expected: #soiu#, prediction: Resposta:#suiu#

verb: #media#, expected: #medeiu#, prediction: Resposta:#meiu#

verb: #kompo#, expected: #koiu#, prediction: Resposta:#koiu#

verb: #po#, expected: #poiu#, prediction: Resposta:#poiu#

verb: #menti#, expected: #mintu#, prediction: Resposta:#mitu#

verb: #tosi#, expected: #tusu#, prediction: Resposta:#topu#

verb: #kobri#, expected: #kubro#, prediction: Resposta:#koru#

verb: #faze#, expected: #fasu#, prediction: Resposta:#fasu#

verb: #mata#, expe

### Experimento 50 - 50

In [59]:
model = load_model('../data/50_50_morar',custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#nota#','#soka#','#loga#','#bate#','#limpa#','#kasa#'])

Resposta:#bOtu#
Resposta:#tOku#
Resposta:#legu#
Resposta:#batu#
Resposta:#lepu#
Resposta:#kasu#


(6, 460)